<a href="https://colab.research.google.com/github/Nikunjmistry22/stackoverflow-data/blob/main/stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local") \
    .appName("StackoverFlow") \
    .config("spark.driver.memory","16g") \
    .getOrCreate()
spark

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "500")

In [ ]:
data=spark.read.format("csv").option("header",True).load("csv_files\\question_tags.csv")
data.show(5)

+---+---------------+
| Id|            Tag|
+---+---------------+
|  1|           data|
|  4|             c#|
|  4|       winforms|
|  4|type-conversion|
|  4|        decimal|
+---+---------------+
only showing top 5 rows



In [ ]:
data.schema

StructType([StructField('Id', StringType(), True), StructField('Tag', StringType(), True)])

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *
w=Window.orderBy(lit(1))

In [ ]:
tags=data.select("Tag").distinct()
tags=tags.orderBy("Tag",ascending=True)
tags.show(5)

+--------+
|     Tag|
+--------+
|       #|
|    #-+-|
|       +|
|-session|
|   -test|
+--------+
only showing top 5 rows



In [ ]:
tag_name=tags.withColumn("Tag_id",row_number().over(w))
tag_name=tag_name.select("Tag_id","Tag")
tag_name.count()

58256

In [ ]:
join_exp=data["Tag"]==tag_name["Tag"]
main_tag=data.join(tag_name,join_exp)
main_tag=main_tag.select(col("Id").alias("question_id"),"Tag_id")
main_tag.show(10)

+-----------+------+
|question_id|Tag_id|
+-----------+------+
|         17|  5401|
|       1041| 42420|
|       1237| 42420|
|       1746| 42420|
|       3232| 42420|
|       4149| 17123|
|       4736| 42420|
|       4870| 42420|
|       4922| 39370|
|       4954| 42420|
+-----------+------+
only showing top 10 rows



In [ ]:
main_tag.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [Id#17 AS question_id#65, Tag_id#41]
   +- SortMergeJoin [Tag#18], [Tag#55], Inner
      :- Sort [Tag#18 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(Tag#18, 500), ENSURE_REQUIREMENTS, [plan_id=626]
      :     +- Filter isnotnull(Tag#18)
      :        +- FileScan csv [Id#17,Tag#18] Batched: false, DataFilters: [isnotnull(Tag#18)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/Nikunj/Desktop/stackoverflow/csv_files/question_tags.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Tag)], ReadSchema: struct<Id:string,Tag:string>
      +- Sort [Tag#55 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(Tag#55, 500), ENSURE_REQUIREMENTS, [plan_id=627]
            +- Project [Tag_id#41, Tag#55]
               +- Filter isnotnull(Tag#55)
                  +- Window [row_number() windowspecdefinition(1 ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), 

In [ ]:
#Read the file
main_data=spark.read.format("csv").option("header",True).load("csv_files\\questions.csv")
main_data.show(5)

+---+--------------------+--------------------+--------------------+-----+-----------+-----------+
| Id|        CreationDate|          ClosedDate|        DeletionDate|Score|OwnerUserId|AnswerCount|
+---+--------------------+--------------------+--------------------+-----+-----------+-----------+
|  1|2008-07-31T21:26:37Z|                  NA|2011-03-28T00:53:47Z|    1|         NA|          0|
|  4|2008-07-31T21:42:52Z|                  NA|                  NA|  458|          8|         13|
|  6|2008-07-31T22:08:08Z|                  NA|                  NA|  207|          9|          5|
|  8|2008-07-31T23:33:19Z|2013-06-03T04:00:25Z|2015-02-11T08:26:40Z|   42|         NA|          8|
|  9|2008-07-31T23:40:59Z|                  NA|                  NA| 1410|          1|         58|
+---+--------------------+--------------------+--------------------+-----+-----------+-----------+
only showing top 5 rows



In [ ]:
#most commom tags doubt
common=data.select("Tag").groupBy("Tag").count()
common.orderBy("count",ascending=False).show()

+-------------+-------+
|          Tag|  count|
+-------------+-------+
|   javascript|1649631|
|         java|1563820|
|          php|1365600|
|           c#|1321027|
|      android|1288338|
|       jquery|1011324|
|       python| 854172|
|         html| 776512|
|          ios| 635227|
|          c++| 627341|
|        mysql| 561375|
|          css| 545587|
|          sql| 436546|
|      asp.net| 400619|
|  objective-c| 338539|
|ruby-on-rails| 325003|
|            c| 306418|
|         .net| 301454|
|       iphone| 274464|
|    angularjs| 267560|
+-------------+-------+
only showing top 20 rows



In [ ]:
#top 3 months where most questions were asked
top_3=main_data.select(month("CreationDate").alias("month"))
top_3.select("month").groupBy("month").count().orderBy("count",ascending=False).show(3)

+-----+-------+
|month|  count|
+-----+-------+
|    1|1536581|
|   10|1489708|
|   11|1475681|
+-----+-------+
only showing top 3 rows



In [ ]:
#unsolved question percentage
condition_count=main_data.select().where(col("AnswerCount")==0).count()
total=main_data.count()
condition_count/total*100

19.220529110272228

In [ ]:
main_data.select("Id","AnswerCount").orderBy("AnswerCount",ascending=True).show(1)

+---+-----------+
| Id|AnswerCount|
+---+-----------+
|630|         -1|
+---+-----------+
only showing top 1 row



In [ ]:
#Deletion top3
top_3=main_data.select(month("DeletionDate").alias("month"))
top_3.select("month").where(month("DeletionDate").isin(1,2,3,4,5,6,7,8,9,10,11,12))\
                            .groupBy("month").count().orderBy("count",ascending=False).show(3)

+-----+------+
|month| count|
+-----+------+
|    8|363930|
|   10|356172|
|    7|349615|
+-----+------+
only showing top 3 rows



In [ ]:
#Weekends Questions vs Non Weekends
from pyspark.sql.functions import dayofweek 
weekend=main_data.withColumn('day_of_week',dayofweek("CreationDate"))
stats=weekend.withColumn("isWeekend",when(col('day_of_week').isin(1,6) ,1).otherwise(0))
stats.groupBy("isWeekend").agg(count('isWeekend')).show()

+---------+----------------+
|isWeekend|count(isWeekend)|
+---------+----------------+
|        1|         4116577|
|        0|        13087247|
+---------+----------------+



In [ ]:
#Active Users
active_users=main_data.filter(~main_data.OwnerUserId.isin('NA'))
active_user_count=active_users.count()
str(active_user_count//100000)+" Lakhs"

'130 Lakhs'

In [ ]:
#Repeated Users
repeated=active_users.groupBy('OwnerUserId').agg(count('Id').alias("Repeated"))
repeat_count=repeated.where(col("Repeated")>1).count()
str(repeat_count//100000)+" Lakhs"

'12 Lakhs'

In [ ]:
#Year Wise stats
top_3=main_data.select(year("CreationDate").alias("year"))
top_3.select("year").groupBy("year").count().orderBy("count",ascending=False).show()

+----+-------+
|year|  count|
+----+-------+
|2016|3201444|
|2015|3105720|
|2014|3040440|
|2013|2759442|
|2012|2065664|
|2011|1445142|
|2010| 820161|
|2009| 394567|
|2017| 300872|
|2008|  70372|
+----+-------+

